## LoanSense Pro – AI-powered Loan Approval Predictor

###  Step 1: Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier


### Step 2: Load the train and test CSVs

In [2]:
train_data = pd.read_csv(r"C:\Users\Asus\Downloads\dataset\loan\train.csv")
test_data = pd.read_csv(r"C:\Users\Asus\Downloads\dataset\loan\test.csv")


### Step 3: Merge data for uniform preprocessing

In [3]:
train_data["is_train"] = 1
test_data["is_train"] = 0
full_data = pd.concat([train_data, test_data], ignore_index=True)
full_data.drop("Loan_ID", axis=1, inplace=True)

### Step 4: Fill missing values

In [4]:
for col in ['Gender', 'Married', 'Dependents', 'Self_Employed', 'Credit_History', 'Loan_Amount_Term']:
    full_data[col] = full_data[col].fillna(full_data[col].mode()[0])
full_data['LoanAmount'] = full_data['LoanAmount'].fillna(full_data['LoanAmount'].median())

### Step 5: Label encode categorical columns

In [5]:
le = LabelEncoder()
cat_cols = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']
for col in cat_cols:
    full_data[col] = le.fit_transform(full_data[col].astype(str))

### Step 6: Separate back train/test

In [6]:
train = full_data[full_data['is_train'] == 1].drop(columns=['is_train'])
test = full_data[full_data['is_train'] == 0].drop(columns=['is_train', 'Loan_Status'])


### Step 7: Define features (X) and target (y)

In [7]:
X = train.drop("Loan_Status", axis=1)
y = train["Loan_Status"].map({'Y': 1, 'N': 0})  # Target encoded here


### Step 8: Split into train and validation sets

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

###  Step 9: Feature Scaling

In [9]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

### Step 10: Train Random Forest

In [10]:
model = RandomForestClassifier(
    n_estimators=1000,
    max_depth=12,
    min_samples_split=5,
    class_weight='balanced',
    random_state=42
)
model.fit(X_train_scaled, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=12,
                       min_samples_split=5, n_estimators=1000, random_state=42)

In [11]:
# ✅ Step 11: Evaluate
y_pred = model.predict(X_val_scaled)
accuracy = accuracy_score(y_val, y_pred)
print(f"✅ Model Accuracy: {accuracy * 100:.2f}%")
print(classification_report(y_val, y_pred))

✅ Model Accuracy: 77.24%
              precision    recall  f1-score   support

           0       0.86      0.42      0.56        43
           1       0.75      0.96      0.85        80

    accuracy                           0.77       123
   macro avg       0.81      0.69      0.70       123
weighted avg       0.79      0.77      0.75       123



### Step 12: Take user input

In [12]:
def get_user_input():
    print("\n🔎 Enter Applicant Details:")
    Gender = input("Gender (Male/Female): ")
    Married = input("Married (Yes/No): ")
    Dependents = input("Dependents (0/1/2/3+): ")
    Education = input("Education (Graduate/Not Graduate): ")
    Self_Employed = input("Self Employed (Yes/No): ")
    ApplicantIncome = float(input("Applicant Monthly Income: "))
    CoapplicantIncome = float(input("Coapplicant Income: "))
    LoanAmount = float(input("Loan Amount 1000X: "))
    Loan_Amount_Term = float(input("Loan Amount Term (in days): "))
    Credit_History = float(input("Credit History (1=Good or 0=Bad): "))
    Property_Area = input("Property Area (Urban/Semiurban/Rural): ")

    user_df = pd.DataFrame({
        "Gender": [Gender],
        "Married": [Married],
        "Dependents": [Dependents],
        "Education": [Education],
        "Self_Employed": [Self_Employed],
        "ApplicantIncome": [ApplicantIncome],
        "CoapplicantIncome": [CoapplicantIncome],
        "LoanAmount": [LoanAmount],
        "Loan_Amount_Term": [Loan_Amount_Term],
        "Credit_History": [Credit_History],
        "Property_Area": [Property_Area]
    })
    return user_df

### Step 13: Preprocess and predict

In [13]:
def preprocess_user_input(user_df):
    for col in cat_cols:
        user_df[col] = le.fit_transform(user_df[col].astype(str))
    return scaler.transform(user_df)

user_input_df = get_user_input()
user_input_processed = preprocess_user_input(user_input_df)
prediction = model.predict(user_input_processed)

print("\n🔮 Prediction Result:")
print("✅ Loan Approved" if prediction[0] == 1 else "❌ Loan Rejected")


🔎 Enter Applicant Details:


Gender (Male/Female):  Male
Married (Yes/No):  Yes
Dependents (0/1/2/3+):  0
Education (Graduate/Not Graduate):  Graduate
Self Employed (Yes/No):  Yes
Applicant Monthly Income:  25000
Coapplicant Income:  2000
Loan Amount 1000X:  10
Loan Amount Term (in days):  128
Credit History (1=Good or 0=Bad):  1
Property Area (Urban/Semiurban/Rural):  Semiurban



🔮 Prediction Result:
✅ Loan Approved
